In [1]:
%reset -f
%load_ext autoreload
%autoreload 2
# import sys
# sys.path.append('/home/saptarshi/Dhruv/Dissertation/ICC/')
from generate_attributions import *
from utils import CONFIG,evaluate_exp,PREFIX,aggregate_auc_or_sum

/home/saptarshi/anaconda3/envs/Dhruv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## German Credit Dataset

In [2]:
config = CONFIG['german']
mlp_path = PREFIX + 'models/german_mlp/german_mlp_seed_20/logs/version_2/checkpoints/epoch=69-step=910.ckpt'
scm_path = PREFIX + 'models/german_nf/german_nf_seed_20/logs/version_1/checkpoints/epoch=4-step=65.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [3]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [4]:
# Load JSON
with open(PREFIX + "attributions/german/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

In [5]:
ge_dict = data["ge_dict"]
evaluation_metrics,evaluation_metrics_per_sample = evaluate_exp(ge_dict,config,mlp_model.to('cpu'))
print(evaluation_metrics)


{'ig': {'pgi': {'k=1': [0.28330445289611816], 'k=2': [0.43073683977127075], 'k=3': [0.4646214246749878], 'k=4': [0.43991902470588684]}, 'pgu': {'k=1': [0.3754216432571411], 'k=2': [0.21531988680362701], 'k=3': [0.0998459905385971], 'k=4': [1.5497953143039922e-08]}}, 'itg': {'pgi': {'k=1': [0.28334569931030273], 'k=2': [0.27840349078178406], 'k=3': [0.33242443203926086], 'k=4': [0.4406302571296692]}, 'pgu': {'k=1': [0.37505313754081726], 'k=2': [0.40922972559928894], 'k=3': [0.36289605498313904], 'k=4': [1.5497953143039922e-08]}}, 'sg': {'pgi': {'k=1': [0.28377193212509155], 'k=2': [0.27876898646354675], 'k=3': [0.3330904245376587], 'k=4': [0.44078755378723145]}, 'pgu': {'k=1': [0.37495994567871094], 'k=2': [0.41081610321998596], 'k=3': [0.36265867948532104], 'k=4': [1.5497953143039922e-08]}}, 'shap': {'pgi': {'k=1': [0.28304028511047363], 'k=2': [0.34970974922180176], 'k=3': [0.4634457528591156], 'k=4': [0.440280556678772]}, 'pgu': {'k=1': [0.37520164251327515], 'k=2': [0.3230814635753

In [6]:
auc_results,auc_show,sum_results,sum_show = aggregate_auc_or_sum(evaluation_metrics_per_sample,config)

# Print results nicely
for method, vals in auc_show.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']}, PGU AUC = {vals['pgu_auc']}")


ig: PGI AUC = 0.41899000212550164 +- 0.003857233410552189, PGU AUC = 0.16762556297083697 +- 0.0014466489587162787
itg: PGI AUC = 0.3242719764759143 +- 0.002468114759564698, PGU AUC = 0.31988412372767927 +- 0.003610294437924551
sg: PGI AUC = 0.3247130434835951 +- 0.002461942787677008, PGU AUC = 0.3203182592925926 +- 0.0036129839476558468
shap: PGI AUC = 0.39160531741877397 +- 0.00352047890285511, PGU AUC = 0.20351038393874962 +- 0.001336818723789011
lime: PGI AUC = 0.34765599783509976 +- 0.002878249359231467, PGU AUC = 0.2910708352861305 +- 0.0030025131990754185
sp_lime: PGI AUC = 0.28698696644355853 +- 0.0020660761284635506, PGU AUC = 0.315446795510749 +- 0.0027872361263103465
pfi: PGI AUC = 0.4000219308460753 +- 0.0032974020240082035, PGU AUC = 0.20457628635844835 +- 0.002372082310172912
icc_topo: PGI AUC = 0.4001388462384542 +- 0.003293728170951571, PGU AUC = 0.20404045618604869 +- 0.0023374039917381842
icc_shap: PGI AUC = 0.4000085904200872 +- 0.003301149364439707, PGU AUC = 0.20432

In [7]:
for method, vals in sum_show.items():
    print(f"{method}: PGI SUM = {vals['pgi_sum']}, PGU SUM = {vals['pgu_sum']}")

ig: PGI SUM = 1.6185816526412964 +- 0.014394131841711828, PGU SUM = 0.6905875205993652 +- 0.004983700990539633
itg: PGI SUM = 1.334803819656372 +- 0.010291891621435852, PGU SUM = 1.1471790075302124 +- 0.012641711188491004
sg: PGI SUM = 1.336418867111206 +- 0.010290892741126144, PGU SUM = 1.1484348773956299 +- 0.012633103748353866
shap: PGI SUM = 1.536476492881775 +- 0.013529716837492796, PGU SUM = 0.7981319427490234 +- 0.005521948909831345
lime: PGI SUM = 1.4039911031723022 +- 0.011660785436248456, PGU SUM = 1.0609631538391113 +- 0.01080649410504174
sp_lime: PGI SUM = 1.1305783987045288 +- 0.007491499589868169, PGU SUM = 1.1786715984344482 +- 0.010590676952556857
pfi: PGI SUM = 1.5616987943649292 +- 0.012742873107958064, PGU SUM = 0.8016570806503296 +- 0.008302444994723854
icc_topo: PGI SUM = 1.5615147352218628 +- 0.012668638810004613, PGU SUM = 0.7996667623519897 +- 0.008202870957774442
icc_shap: PGI SUM = 1.561395287513733 +- 0.012738282262484016, PGU SUM = 0.8009902238845825 +- 0.00

## Lung Cancer Dataset

In [8]:
config = CONFIG['cancer']
mlp_path = PREFIX + 'models/cancer_mlp/cancer_mlp_seed_20/logs/version_1/checkpoints/epoch=62-step=2394.ckpt'
scm_path = PREFIX + 'models/cancer_nf/cancer_nf_seed_30/logs/version_1/checkpoints/epoch=143-step=5472.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [9]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [10]:
# Load JSON
with open(PREFIX +"attributions/cancer/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

In [11]:
ge_dict = data["ge_dict"]
evaluation_metrics,evaluation_metrics_per_sample = evaluate_exp(ge_dict,config,mlp_model.to('cpu'))
print(evaluation_metrics)


{'ig': {'pgi': {'k=1': [0.29143428802490234], 'k=2': [0.3937326967716217], 'k=3': [0.41322463750839233], 'k=4': [0.43354395031929016], 'k=5': [0.4576254189014435], 'k=6': [0.4592629075050354], 'k=7': [0.46727004647254944]}, 'pgu': {'k=1': [0.4178701341152191], 'k=2': [0.37357819080352783], 'k=3': [0.33473873138427734], 'k=4': [0.2627333402633667], 'k=5': [0.18659012019634247], 'k=6': [0.13730411231517792], 'k=7': [1.732276899701901e-08]}}, 'itg': {'pgi': {'k=1': [0.21446695923805237], 'k=2': [0.2688705027103424], 'k=3': [0.31020572781562805], 'k=4': [0.36356714367866516], 'k=5': [0.38598161935806274], 'k=6': [0.41745662689208984], 'k=7': [0.46740299463272095]}, 'pgu': {'k=1': [0.4374137222766876], 'k=2': [0.4198490381240845], 'k=3': [0.3992440700531006], 'k=4': [0.3653901517391205], 'k=5': [0.3590729236602783], 'k=6': [0.29125526547431946], 'k=7': [1.732276899701901e-08]}}, 'sg': {'pgi': {'k=1': [0.21434463560581207], 'k=2': [0.29877060651779175], 'k=3': [0.32868826389312744], 'k=4': [

In [12]:
auc_results,auc_show,sum_results,sum_show = aggregate_auc_or_sum(evaluation_metrics_per_sample,config)

# Print results nicely
for method, vals in auc_show.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']}, PGU AUC = {vals['pgu_auc']}")


ig: PGI AUC = 0.422790296577538 +- 0.0054927717787144935, PGU AUC = 0.250646587257377 +- 0.003287625287743739
itg: PGI AUC = 0.3478360978136253 +- 0.0054580318121808845, PGU AUC = 0.3422530595640031 +- 0.0037885500920092053
sg: PGI AUC = 0.3592782298134019 +- 0.005558009324140253, PGU AUC = 0.3302283215681867 +- 0.0037889343833292714
shap: PGI AUC = 0.36266939613347254 +- 0.005436442444042508, PGU AUC = 0.34296469901888893 +- 0.00378475407553397
lime: PGI AUC = 0.38093441364976266 +- 0.004243713167375744, PGU AUC = 0.30300729491618567 +- 0.0048798620798054144
sp_lime: PGI AUC = 0.34792651644390493 +- 0.0049058731425352395, PGU AUC = 0.3103901141221609 +- 0.003797452427879368
pfi: PGI AUC = 0.41600437760767006 +- 0.005128989905570022, PGU AUC = 0.2770108573940686 +- 0.004273577566147368
icc_topo: PGI AUC = 0.42503153905065527 +- 0.005425998826980188, PGU AUC = 0.2543862634777583 +- 0.0033561569825395553
icc_shap: PGI AUC = 0.42044343039807347 +- 0.005222422671933947, PGU AUC = 0.2640472

In [13]:
for method, vals in sum_show.items():
    print(f"{method}: PGI SUM = {vals['pgi_sum']}, PGU SUM = {vals['pgu_sum']}")

ig: PGI SUM = 2.9160940647125244 +- 0.03714423472746657, PGU SUM = 1.7128145694732666 +- 0.022890291817799488
itg: PGI SUM = 2.4279513359069824 +- 0.03777311198836475, PGU SUM = 2.2722253799438477 +- 0.025763955831085712
sg: PGI SUM = 2.4962244033813477 +- 0.03837138014674112, PGU SUM = 2.200388193130493 +- 0.02573974400422412
shap: PGI SUM = 2.516663074493408 +- 0.03763871566581273, PGU SUM = 2.276855230331421 +- 0.02577208808489189
lime: PGI SUM = 2.6648764610290527 +- 0.029704220168083953, PGU SUM = 2.027061939239502 +- 0.03257286661483866
sp_lime: PGI SUM = 2.383112668991089 +- 0.03348778653497374, PGU SUM = 2.0904858112335205 +- 0.025913443786823222
pfi: PGI SUM = 2.875715732574463 +- 0.0349828169600137, PGU SUM = 1.8704900741577148 +- 0.028841717028092153
icc_topo: PGI SUM = 2.929060220718384 +- 0.036736378595597384, PGU SUM = 1.7351601123809814 +- 0.02327675880814974
icc_shap: PGI SUM = 2.902144193649292 +- 0.03555577828428349, PGU SUM = 1.7927380800247192 +- 0.02745190463254919

## Synthetic Data

In [14]:
config = CONFIG['syn']
mlp_path = PREFIX + 'models/syn_mlp/syn_mlp_seed_20/logs/version_3/checkpoints/epoch=12-step=416.ckpt'
scm_path = PREFIX + 'models/syn_nf/syn_nf_seed_20/logs/version_1/checkpoints/epoch=3-step=128.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [15]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [16]:
# Load JSON
with open(PREFIX +"attributions/syn/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

In [17]:
ge_dict = data["ge_dict"]
evaluation_metrics,evaluation_metrics_per_sample = evaluate_exp(ge_dict,config,mlp_model.to('cpu'))
print(evaluation_metrics)


{'ig': {'pgi': {'k=1': [0.029673971235752106], 'k=2': [0.04707258939743042], 'k=3': [0.06591856479644775]}, 'pgu': {'k=1': [0.04495923966169357], 'k=2': [0.024510297924280167], 'k=3': [1.8179417082819782e-08]}}, 'itg': {'pgi': {'k=1': [0.025141827762126923], 'k=2': [0.044914621859788895], 'k=3': [0.06602416932582855]}, 'pgu': {'k=1': [0.04924170672893524], 'k=2': [0.029749631881713867], 'k=3': [1.8179417082819782e-08]}}, 'sg': {'pgi': {'k=1': [0.02971023879945278], 'k=2': [0.04715872183442116], 'k=3': [0.06593552976846695]}, 'pgu': {'k=1': [0.04489047825336456], 'k=2': [0.024538516998291016], 'k=3': [1.8179417082819782e-08]}}, 'shap': {'pgi': {'k=1': [0.029754795134067535], 'k=2': [0.047038525342941284], 'k=3': [0.06606392562389374]}, 'pgu': {'k=1': [0.044876210391521454], 'k=2': [0.024499068036675453], 'k=3': [1.8179417082819782e-08]}}, 'lime': {'pgi': {'k=1': [0.029794491827487946], 'k=2': [0.047061506658792496], 'k=3': [0.06594599038362503]}, 'pgu': {'k=1': [0.04484673961997032], 'k

In [18]:
auc_results,auc_show,sum_results,sum_show = aggregate_auc_or_sum(evaluation_metrics_per_sample,config)

# Print results nicely
for method, vals in auc_show.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']}, PGU AUC = {vals['pgu_auc']}")


ig: PGI AUC = 0.04743443028690914 +- 0.00013928067187151286, PGU AUC = 0.023494963405731446 +- 0.0001143035625090633
itg: PGI AUC = 0.04524880940715472 +- 0.00016779264744936194, PGU AUC = 0.02718524675583467 +- 7.554990755793751e-05
sg: PGI AUC = 0.04749080207509299 +- 0.00014033068506982645, PGU AUC = 0.02349188173810641 +- 0.00011453199251605268
shap: PGI AUC = 0.047473943273847304 +- 0.0001399436251303109, PGU AUC = 0.023468590681441127 +- 0.00011388033015324986
lime: PGI AUC = 0.04746587418330212 +- 0.00013850168800928604, PGU AUC = 0.023547237147577105 +- 0.0001138199178110556
sp_lime: PGI AUC = 0.04736627360340208 +- 0.00014469930311302844, PGU AUC = 0.024435156686231495 +- 0.00010216998120572126
pfi: PGI AUC = 0.0485636753278474 +- 0.00013270960120746213, PGU AUC = 0.023844883123723168 +- 0.00010875367478809509
icc_topo: PGI AUC = 0.047501243685061735 +- 0.00014022401694885426, PGU AUC = 0.023440039810181286 +- 0.00011441043975770566
icc_shap: PGI AUC = 0.0474537610138456 +- 0.

In [19]:
for method, vals in sum_show.items():
    print(f"{method}: PGI SUM = {vals['pgi_sum']}, PGU SUM = {vals['pgu_sum']}")

ig: PGI SUM = 0.14266513288021088 +- 0.0004090698789623142, PGU SUM = 0.06946955621242523 +- 0.0003170963803254705
itg: PGI SUM = 0.13608062267303467 +- 0.0004909883975298063, PGU SUM = 0.0789913535118103 +- 0.00021823882207849314
sg: PGI SUM = 0.14280447363853455 +- 0.0004053969280981332, PGU SUM = 0.06942901015281677 +- 0.0003165815748359825
shap: PGI SUM = 0.14285725355148315 +- 0.0004081791209853773, PGU SUM = 0.06937529146671295 +- 0.00031428911253273767
lime: PGI SUM = 0.14280197024345398 +- 0.00040127236278756094, PGU SUM = 0.06951785832643509 +- 0.00031656121454502083
sp_lime: PGI SUM = 0.14000096917152405 +- 0.0004460296052731675, PGU SUM = 0.072452612221241 +- 0.0002778006765797092
pfi: PGI SUM = 0.14490222930908203 +- 0.00039074671970828524, PGU SUM = 0.07011809945106506 +- 0.0003029368150256593
icc_topo: PGI SUM = 0.14286306500434875 +- 0.0004094366683683032, PGU SUM = 0.06928133964538574 +- 0.00031674708061853353
icc_shap: PGI SUM = 0.14271573722362518 +- 0.000406086060261

## Test

In [2]:
config = CONFIG['syn']
mlp_path = '/home/dhruv/Files/Thesis/Dissertation/Code/models/syn_ann/syn_ann_seed_10/logs/version_0/checkpoints/epoch=161-step=5184.ckpt'
scm_path = '/home/dhruv/Files/Thesis/Dissertation/Code/models/syn_nf/syn_nf_seed_10/logs/version_0/checkpoints/epoch=19-step=640.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_ann'],classification=False))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))
ge_dict, evaluation_metrics, time_dict, global_explanations = generate_global_exps(config,mlp_model,scm_model)

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [3]:
ge_dict

{'ig': {'W': [0.008685602410414437],
  'Z': [0.5541948194138167],
  'X': [0.4371195781757688]},
 'itg': {'W': [0.008485306748260027],
  'Z': [0.497316907775009],
  'X': [0.49419778547673104]},
 'sg': {'W': [0.13136424456458626],
  'Z': [0.3860659244300918],
  'X': [0.48256983100532197]},
 'shap': {'W': [0.5803365821763931],
  'Z': [0.3543153621348831],
  'X': [0.06534805568872387]},
 'lime': {'W': [0.2837432524861816],
  'Z': [0.293367905107146],
  'X': [0.4228888424066725]},
 'sp_lime': {'X': [0.41602805243460733],
  'W': [0.2947343644233361],
  'Z': [0.2892375831420565]},
 'pfi': {'W': [0.2555289566516876],
  'Z': [0.2728649377822876],
  'X': [0.47160604596138]},
 'icc_topo': {'W': [0.07992755621671677],
  'Z': [0.6083893179893494],
  'X': [0.31013962626457214]},
 'icc_shap': {'W': [0.1287594437599182],
  'Z': [0.6630550622940063],
  'X': [0.20664207637310028]}}

In [9]:
generate_attr_plot(global_explanations,config)

In [7]:
# Load JSON
with open("/home/dhruv/Files/Thesis/Dissertation/Code/attributions/syn/attribution.json", "r") as f:
    data = json.load(f)

# Call the function
summary = compute_pgu_pgi_sums(data["evaluation_metrics"])

# Display the results
for method, scores in summary.items():
    print(f"{method}: PGU Sum = {scores['pgu_sum']:.6f}, PGI Sum = {scores['pgi_sum']:.6f}")


ig: PGU Sum = 0.128419, PGI Sum = 0.000436
itg: PGU Sum = 0.128419, PGI Sum = 0.000436
sg: PGU Sum = 0.128418, PGI Sum = 0.000437
shap: PGU Sum = 0.128420, PGI Sum = 0.000435
lime: PGU Sum = 0.128418, PGI Sum = 0.000437
sp_lime: PGU Sum = 0.128420, PGI Sum = 0.000435
pfi: PGU Sum = 0.128418, PGI Sum = 0.000437
icc_topo: PGU Sum = 0.128419, PGI Sum = 0.000436
icc_shap: PGU Sum = 0.128419, PGI Sum = 0.000436
